# core

> This is where users start to interact with the underlying LLM and agents.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
# install dependencies
# %pip install replicate
# %pip install gradio
# %pip install langchain

In [ ]:
#| export
# import langchain

import os
import llm_QA


/Volumes/SSD2TB/Google Drive/Work/MachineLearning/SciAI/nbs/llm_QA.py:197: SyntaxWarning: invalid escape sequence '\<'
  """


In [ ]:
#| export

model = llm_QA.LLM("meta-llama", "TogetherAI")

In [ ]:
#| hide
# interact with the model directly to test
messages = [
  {
    "role": "user",
    "content": "what are large language models?"
  }
]

response = model.get_llm_response(messages)
print(response)

In [ ]:
#| hide
# interact with the model directly to test multimodal inputs
from PIL import Image
import matplotlib.pyplot as plt

def display_local_image(image_path):
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')
    plt.show()

display_local_image("images/a_colorful_llama_doing_ai_programming.jpeg")



In [ ]:
#| export
# A chatbot that allows users to interact with an LLM
import gradio as gr

def taskRouter(input, history):
    prompt = input["text"]
    files = input["files"]
    messages = [history,
        {
            "role": "user",
            "content": prompt
        }
    ]

    response = model.get_llm_response(messages)
    #history = [messages, 
    #    {
    #        "role": "assistant",
    #        "content": response
    #    }
    #]
    return response

def vote(data: gr.LikeData):
    if data.liked:
        print("You upvoted this response: " + data.value["value"])
    else:
        print("You downvoted this response: " + data.value["value"])

with gr.Blocks() as sciChat:
    chatbot = gr.Chatbot(
        height=300,
        type="messages",
        placeholder="<strong>Your Personal Science AI Assistant</strong><br>Ask Me Anything")
    chatbot.like(vote, None, None)
    gr.ChatInterface(
        fn=taskRouter,
        type="messages",
        chatbot=chatbot,
        textbox=gr.MultimodalTextbox(placeholder="How can I help?", container=False, scale=7),
        title="SciAI Assistant",
        description="Ask me any question or carry out a task",
        theme="soft",
        examples=[{"text": "Hello", "files": []},{"text": "What do you see in this image?", "files": []}],
        cache_examples=True,
        multimodal=True
    )
    
sciChat.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
Using cache from '/Volumes/SSD2TB/Google Drive/Work/MachineLearning/SciAI/nbs/.gradio/cached_examples/11' directory. If method or examples have changed since last caching, delete this folder to clear cache.


Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Traceback (most recent call last):
  File "/Users/danc/anaconda3/envs/llm/lib/python3.12/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/danc/anaconda3/envs/llm/lib/python3.12/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/danc/anaconda3/envs/llm/lib/python3.12/site-packages/gradio/blocks.py", line 2015, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/danc/anaconda3/envs/llm/lib/python3.12/site-packages/gradio/blocks.py", line 1560, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/danc/anaconda3/envs/llm/lib/python3.12/site-packages/gradio/utils.py", line 832, in async_wrapper
    response = awai

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()